In [1]:
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver # driver de selenium

import time   # para manejar tiempo

In [2]:
from selenium.webdriver.chrome.options import Options   # opciones del driver

from selenium.webdriver.common.by import By   # para buscar segun tag, id, class, etc...

from selenium.webdriver.support.ui import WebDriverWait   # es para esperar

from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...

from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click

from selenium.webdriver.common.keys import Keys  # manejar teclas

In [3]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

#### 1. Conecto con la página WEB

In [4]:
PATH = 'driver/chromedriver'

driver = webdriver.Chrome()   # abre una ventana de chrome 
                                # una vez que he abierto una ventana, puedo trabajar siempre sobre esta misma.

In [5]:
url = 'https://www.entradas.com/city/madrid-370/conciertos-y-festivales-85/'

driver.get(url)

Conecto con la Web de Entradas.com para trabajar sobre esta, y hacer scraping de los datos que quiero. Utilizaré el método de Scrapeo de selenium.

#### 2. Aceptar las cookies 

- Lo primero que hago es darle a Aceptas las cookies del sitio web. 
- Para ello tengo un problema inicial, ya que está categorizada como una pestaña "shadow" por lo que cuando busco la opción de hacer click al botón de aceptar, aparece como una pestaña / lista vacía. 

In [7]:
div_shadow = driver.find_element(By.XPATH, '//div[@class="cmpwrapper"]')

In [8]:
# Check if the element has a Shadow DOM using JavaScript
has_shadow_dom = driver.execute_script('return arguments[0].shadowRoot !== null', div_shadow)

if has_shadow_dom:
    print("Element has a Shadow DOM")
else:
    print("Element does not have a Shadow DOM")

Element has a Shadow DOM


In [9]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', div_shadow)

In [10]:
shadow_root.find_elements(By.CSS_SELECTOR, '#cmpwelcomebtnyes > a')[0].click()

- Una vez consigo traspasar la barrera del 'Shadow DOM', entonces puedo comenzar con el scrapeo de las tablas que necesito para mi estudio. 

#### 3. Empezamos el scrapeo por eventos en tabla

In [11]:
tabla = '//product-group-item'

In [12]:
table = driver.find_elements(By.XPATH, tabla)
table

[<selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_27")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_28")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_29")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_30")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_31")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_32")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787

In [13]:
type(table)

list

In [14]:
elementos = driver.find_elements(By.XPATH, '//*[@id="list-item-0"]')
elementos

[<selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_60")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_61")>]

In [15]:
type(elementos)

list

In [16]:
elementos[0].text

'Cariño\nInverfest 2024\n13/01/20241 Evento\n desde 19,80 €\nMostrar 1 Evento'

In [24]:
elementos = elementos[0].text.split('\n')
elementos

['Cariño',
 'Inverfest 2024',
 '13/01/20241 Evento',
 ' desde 19,80 €',
 'Mostrar 1 Evento']

In [21]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_73")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec388d3ab1d904b59c02b2b16787f60b", element="4980F7304F6A84BC7CC2CED8CFCFA8B6_element_47")>]

In [60]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    
    data.append([artists, dates])

for entry in data:
    print(entry)

[['Cariño', 'Iván Ferreiro', 'The World Of Hans Zimmer', 'Triángulo de Amor Bizarro', 'Nico B', 'Cala Vento', 'Sidonie', 'Belako - Sigo regando', 'Biznaga', 'Morreo - Alegría Club!', 'Diego el Cigala - Obras Maestras', 'Symphonic Rhapsody of Queen', 'Sexy Zebras', 'Biografía - Rafa Sánchez, de la unión', 'OBK', 'El Consorcio - Toda una vida', 'Rock en Familia - Hijos del Rock and Roll', 'Rock en Familia - I Love Rock and Roll', 'Rock en Familia - Queenmanía', 'Rock en Familia - I Love Heavy Metal'], ['13/01/2024', '1 Evento', '13/01/2024', '1 Evento', '24/04/2024 ─ 29/04/2024', '2 eventos', '29/12/2023', '1 Evento', '14/01/2024', '1 Evento', '01/12/2023', '1 Evento', '12/01/2024 ─ 14/01/2024', '2 eventos', '20/01/2024', '1 Evento', '26/01/2024', '1 Evento', '18/01/2024', '1 Evento', '11/12/2023 ─ 18/12/2023', '2 eventos', '16/01/2024', '1 Evento', '01/02/2024', '1 Evento', '11/12/2023', '1 Evento', '13/01/2024 ─ 27/01/2024', '3 eventos', '27/11/2023 ─ 28/11/2023', '2 eventos', '05/11/2

In [71]:
artistas = entry[0]
artistas

['Cariño',
 'Iván Ferreiro',
 'The World Of Hans Zimmer',
 'Triángulo de Amor Bizarro',
 'Nico B',
 'Cala Vento',
 'Sidonie',
 'Belako - Sigo regando',
 'Biznaga',
 'Morreo - Alegría Club!',
 'Diego el Cigala - Obras Maestras',
 'Symphonic Rhapsody of Queen',
 'Sexy Zebras',
 'Biografía - Rafa Sánchez, de la unión',
 'OBK',
 'El Consorcio - Toda una vida',
 'Rock en Familia - Hijos del Rock and Roll',
 'Rock en Familia - I Love Rock and Roll',
 'Rock en Familia - Queenmanía',
 'Rock en Familia - I Love Heavy Metal']

In [75]:
fecha_eventos = entry[1]
fecha_eventos

['13/01/2024',
 '1 Evento',
 '13/01/2024',
 '1 Evento',
 '24/04/2024 ─ 29/04/2024',
 '2 eventos',
 '29/12/2023',
 '1 Evento',
 '14/01/2024',
 '1 Evento',
 '01/12/2023',
 '1 Evento',
 '12/01/2024 ─ 14/01/2024',
 '2 eventos',
 '20/01/2024',
 '1 Evento',
 '26/01/2024',
 '1 Evento',
 '18/01/2024',
 '1 Evento',
 '11/12/2023 ─ 18/12/2023',
 '2 eventos',
 '16/01/2024',
 '1 Evento',
 '01/02/2024',
 '1 Evento',
 '11/12/2023',
 '1 Evento',
 '13/01/2024 ─ 27/01/2024',
 '3 eventos',
 '27/11/2023 ─ 28/11/2023',
 '2 eventos',
 '05/11/2023 ─ 10/03/2024',
 '2 eventos',
 '03/12/2023',
 '1 Evento',
 '17/12/2023 ─ 14/04/2024',
 '2 eventos',
 '19/11/2023 ─ 24/03/2024',
 '2 eventos']

In [77]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['13/01/2024', '13/01/2024', '24/04/2024 ─ 29/04/2024', '29/12/2023', '14/01/2024', '01/12/2023', '12/01/2024 ─ 14/01/2024', '20/01/2024', '26/01/2024', '18/01/2024', '11/12/2023 ─ 18/12/2023', '16/01/2024', '01/02/2024', '11/12/2023', '13/01/2024 ─ 27/01/2024', '27/11/2023 ─ 28/11/2023', '05/11/2023 ─ 10/03/2024', '03/12/2023', '17/12/2023 ─ 14/04/2024', '19/11/2023 ─ 24/03/2024']
['1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '3 eventos', '2 eventos', '2 eventos', '1 Evento', '2 eventos', '2 eventos']


In [78]:
import pandas as pd

In [81]:
datos = pd.DataFrame({'Artista': artistas, 'Fechas': fechas, 'Eventos': num_eventos})

datos

,Artista,Fechas,Eventos
0,Cariño,13/01/2024,1 Evento
1,Iván Ferreiro,13/01/2024,1 Evento
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento
4,Nico B,14/01/2024,1 Evento
5,Cala Vento,01/12/2023,1 Evento
6,Sidonie,12/01/2024 ─ 14/01/2024,2 eventos
7,Belako - Sigo regando,20/01/2024,1 Evento
8,Biznaga,26/01/2024,1 Evento
9,Morreo - Alegría Club!,18/01/2024,1 Evento


In [25]:
type(elementos)

list

In [34]:
todo = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
todo


[<selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_83")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_81")>]

In [39]:
tabla_def = todo[1]
tabla_def

<selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_81")>

In [52]:
datos_imp = tabla_def.find_elements(By.XPATH, '//product-group-item[@class="hydrated"]')
datos_imp

[<selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_26")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_27")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_28")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_29")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_30")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f7366", element="E42A3DFE0B7A2888FD9668A68E292070_element_31")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15e9bb53320375a102dd1348bc7f

In [54]:
datos_imp[0].text

'Cariño\nInverfest 2024\n13/01/20241 Evento\n desde 19,80 €\nMostrar 1 Evento\nMadrid, sáb. 13/01/2024, 21:00\nCariño\n Entradas'

In [56]:
filas = []

# Iterar sobre los elementos en 'datos_imp'
for elem in datos_imp:
    # Obtener el texto de cada elemento y agregarlo a la lista de filas
    filas.append(elem.text)
    
filas

['Cariño\nInverfest 2024\n13/01/20241 Evento\n desde 19,80 €\nMostrar 1 Evento\nMadrid, sáb. 13/01/2024, 21:00\nCariño\n Entradas',
 'Iván Ferreiro\nInverfest 2024\n13/01/20241 Evento\n desde 38,50 €\nMostrar 1 Evento\nMadrid, sáb. 13/01/2024, 20:30\nIVÁN FERREIRO\n Entradas',
 'The World Of Hans Zimmer\n"A New Dimension" - European World Tour 2024\n24/04/2024 ─ 29/04/20242 eventos\nEntradas VIP\n  desde 55,00 €\nMostrar los 2 eventos\nMadrid, lun. 29/04/2024, 20:00\nThe World Of Hans Zimmer: A New Dimension\n  Entradas',
 'Triángulo de Amor Bizarro\nInverfest 2024\n29/12/20231 Evento\n desde 19,80 €\nMostrar 1 Evento\nMadrid, vie. 29/12/2023, 21:00\nTriángulo de Amor Bizarro\n Entradas',
 'Nico B\nInverfest 2024\n14/01/20241 Evento\n desde 13,50 €\nMostrar 1 Evento\nMadrid, dom. 14/01/2024, 19:30\nNico B\n Entradas',
 'Cala Vento\nInverfest 2024\n01/12/20231 Evento\n desde 19,80 €\nMostrar 1 Evento\nMadrid, vie. 01/12/2023, 21:00\nCala Vento\n Entradas',
 'Sidonie\nInverfest 2024\n12/

In [58]:
# Crear una lista para almacenar las filas
filas = []

# Iterar sobre los elementos en 'datos_imp'
for elem in datos_imp:
    # Obtener el texto de cada elemento y dividirlo en una lista
    fila = elem.text.split('\n')
    # Agregar la lista resultante a 'filas'
    filas.append(fila)

filas

[['Cariño',
  'Inverfest 2024',
  '13/01/20241 Evento',
  ' desde 19,80 €',
  'Mostrar 1 Evento',
  'Madrid, sáb. 13/01/2024, 21:00',
  'Cariño',
  ' Entradas'],
 ['Iván Ferreiro',
  'Inverfest 2024',
  '13/01/20241 Evento',
  ' desde 38,50 €',
  'Mostrar 1 Evento',
  'Madrid, sáb. 13/01/2024, 20:30',
  'IVÁN FERREIRO',
  ' Entradas'],
 ['The World Of Hans Zimmer',
  '"A New Dimension" - European World Tour 2024',
  '24/04/2024 ─ 29/04/20242 eventos',
  'Entradas VIP',
  '  desde 55,00 €',
  'Mostrar los 2 eventos',
  'Madrid, lun. 29/04/2024, 20:00',
  'The World Of Hans Zimmer: A New Dimension',
  '  Entradas'],
 ['Triángulo de Amor Bizarro',
  'Inverfest 2024',
  '29/12/20231 Evento',
  ' desde 19,80 €',
  'Mostrar 1 Evento',
  'Madrid, vie. 29/12/2023, 21:00',
  'Triángulo de Amor Bizarro',
  ' Entradas'],
 ['Nico B',
  'Inverfest 2024',
  '14/01/20241 Evento',
  ' desde 13,50 €',
  'Mostrar 1 Evento',
  'Madrid, dom. 14/01/2024, 19:30',
  'Nico B',
  ' Entradas'],
 ['Cala Vento',


In [65]:
filas[13]

['Sexy Zebras',
 '01/02/20241 Evento',
 ' desde 18,00 €',
 'Mostrar 1 Evento',
 'Madrid, jue. 01/02/2024, 21:00',
 'Sexy Zebras',
 ' Entradas']

In [66]:
for elemento in datos_imp:
    # Obtener el texto de cada elemento y dividirlo en una lista
    fila = elemento.text.split('\n')
    # Quedarse solo con los primeros 5 elementos de la lista
    fila = fila[:5]
    # Agregar la lista resultante a 'filas'
    filas.append(fila)

KeyboardInterrupt: 

In [30]:
# Supongamos que tienes 'table' y 'elementos' como los describiste

# Crear una lista para almacenar las filas de la nueva tabla
tabla_nueva = []

# Iterar a través de 'table'
for fila in table:
    # Encontrar 'elementos' en la fila actual
    elementos_fila = fila.find_elements(By.XPATH, '//product-group-item')
    
    # Verificar si 'elementos' está presente en la fila
    if elementos_fila:
        # Extraer el primer elemento de 'elementos' en la fila y agregarlo a la tabla nueva
        primer_elemento = elementos_fila[0].text.split('\n')
        tabla_nueva.append([primer_elemento])

# Ahora 'tabla_nueva' contiene el primer elemento de cada fila


In [31]:
tabla_nueva

[[['Cariño',
   'Inverfest 2024',
   '13/01/20241 Evento',
   ' desde 19,80 €',
   'Mostrar 1 Evento',
   'Madrid, sáb. 13/01/2024, 21:00',
   'Cariño',
   ' Entradas']],
 [['Cariño',
   'Inverfest 2024',
   '13/01/20241 Evento',
   ' desde 19,80 €',
   'Mostrar 1 Evento',
   'Madrid, sáb. 13/01/2024, 21:00',
   'Cariño',
   ' Entradas']],
 [['Cariño',
   'Inverfest 2024',
   '13/01/20241 Evento',
   ' desde 19,80 €',
   'Mostrar 1 Evento',
   'Madrid, sáb. 13/01/2024, 21:00',
   'Cariño',
   ' Entradas']],
 [['Cariño',
   'Inverfest 2024',
   '13/01/20241 Evento',
   ' desde 19,80 €',
   'Mostrar 1 Evento',
   'Madrid, sáb. 13/01/2024, 21:00',
   'Cariño',
   ' Entradas']],
 [['Cariño',
   'Inverfest 2024',
   '13/01/20241 Evento',
   ' desde 19,80 €',
   'Mostrar 1 Evento',
   'Madrid, sáb. 13/01/2024, 21:00',
   'Cariño',
   ' Entradas']],
 [['Cariño',
   'Inverfest 2024',
   '13/01/20241 Evento',
   ' desde 19,80 €',
   'Mostrar 1 Evento',
   'Madrid, sáb. 13/01/2024, 21:00',
   'C